# NLP 실습 #2: N-gram Language model
황순원의 '소나기' 로 'kenlm'을 이용하여 n-gram language model 학습하고 사용해보기
</br>
</br>
* 준비: Kenlm 설치

* Step 1: Data preprocessing - tokenization

* Step 2: Language Model 생성 (Probability Table을 생성)

* Step 3: Binary file로 Model 변환

* Step 4: Model을 활용하여 시퀀스 확률 찾기

* Step 5: 주어진 시퀀스에서 다음 형태소 예측

* Step 6: 주어진 시퀀스로 시작하는 문장을 예측

* Step 7: 다른 말뭉치로 학습한 모델과 비교

</br>
</br>

## 준비: konlpy와 kenlm library 설치

In [1]:
from collections import defaultdict
import math
from konlpy.tag import Kkma
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home2/jyoh96/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# 1. konlpy의 Kkma 형태소 분석기 예제

tokenizer = Kkma()
tokenizer.morphs("이 문장이 형태소 단위로 잘 출력되나요?")

['이', '문장', '이', '형태소', '단위', '로', '잘', '출력', '되', '나요', '?']

In [3]:
# 2. nltk의 tokenizer 예제 

sent_tokenize('입력이 문장 단위로 잘 구분되는지 확인해보세요. sent_tokenize는 한 line을 받아서 마침표를 기준으로 여러 문장으로 분리해주는 함수입니다! 이 문장은 세번째 문장입니다. 이 문장은 네번째 문장입니다!')

['입력이 문장 단위로 잘 구분되는지 확인해보세요.',
 'sent_tokenize는 한 line을 받아서 마침표를 기준으로 여러 문장으로 분리해주는 함수입니다!',
 '이 문장은 세번째 문장입니다.',
 '이 문장은 네번째 문장입니다!']

In [5]:
# 3. kenlm 설치

!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mkdir -p ./kenlm/build
%cd ./kenlm/build
!cmake ..
!make -j 4
!pip install https://github.com/kpu/kenlm/archive/master.zip
%cd /home/jyoh96/ch2-ngram_lm
#%cd /home/ch2_ngram_language_model

--2021-11-13 18:54:01--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
ERROR: cannot verify kheafield.com's certificate, issued by ‘CN=R3,O=Let's Encrypt,C=US’:
  Issued certificate has expired.
To connect to kheafield.com insecurely, use `--no-check-certificate'.

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now
/home2/jyoh96/ch2-ngram_lm/kenlm/build/kenlm/build
CMake Error: The source directory "/home2/jyoh96/ch2-ngram_lm/kenlm/build/kenlm" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No targets specified and no makefile found.  Stop.
  Using cached https://github.com/kpu/kenlm/archive/master.zip
You should consider upgrading via the '/home2/jyoh96/anaconda3/bin/python -m pip install --upgrade pip' command.
[Errno 2] No such f

## Step 1: Data preprocessing - tokenization
&emsp;&emsp; sonagi.doc의 내용을 문장 단위로 분리.

</br> &emsp;&emsp; 각 문장을 kkma 분석기로 형태소 단위로 분리.

</br> &emsp;&emsp; 형태소로 분석된 문장들과 vocab들을 sonagi.txt, sonagi.voc 파일로 작성.


In [ ]:
# 4. sonagi.doc을 읽어들임. 

lines = []
with open('sonagi.doc', 'rt', encoding='utf-8') as f:
    for line in f:
        lines.append(line)

In [ ]:
# 5. nltk의 sent_tokenize를 이용하여 sonagi.doc의 내용을 문장 단위로 분리
sentences = []
for line in lines:
    sentences.extend(sent_tokenize(line))
    
print(sentences[0:10])

In [ ]:
# 6. Kkma tokenizer로 sentences 내용을 형태소 단위로 분리

morphs_lst = []
for sentence in sentences:
    temp = tokenizer.morphs(sentence)
    morphs_lst.append(temp)

print(morphs_lst[0:10])

In [ ]:
# 7. 형태소 분석된 결과물 확인

print(sentences[0])
print(morphs_lst[0])
print()
print(sentences[1])
print(morphs_lst[1])

In [ ]:
# 8. 형태소 분석된 각 sentence들을 한 줄에 한 문장씩 sonagi.txt 파일로 작성

with open('sonagi.txt', 'wt', encoding='utf-8') as f:
    for morphs in morphs_lst:
        f.write(" ".join(morphs).strip())
        f.write('\n')

In [ ]:
# 9. 형태소 분석된 각 sentence의 형태소들을 전부 set에 집어넣어 중복을 삭제, set에 들어있는 vocab을 한 줄에 한 형태소씩 .voc 파일로 작성

vocab = set()
for morphs in morphs_lst:
    for t in morphs:
        vocab.add(t)

print(vocab)

with open('sonagi.voc', 'wt', encoding='utf-8') as f:
    for line in vocab:
        f.write(line.strip())
        f.write('\n')

In [ ]:
! ls

## Step 2: Language model 생성


### &emsp;>> ! kenlm/build/bin/lmplz -o [N] < [txt file] > [model file]
&emsp;&emsp;[N] : 사용할 모델의 n-gram

</br> &emsp;&emsp;[txt file] : 학습(counting)에 사용할 문서 경로

</br> &emsp;&emsp;[model file] : 생성할 모델 파일명

In [ ]:
# 10. kenlm 모델 build

! kenlm/build/bin/lmplz -o 3 <sonagi.txt> sonagi.arpa

In [ ]:
!ls

## Step 3: Binary file로 Model 변환
### &emsp;>> ! kenlm/build/bin/build_binary [model file] [bin model file]

&emsp;&emsp;[N] : n-gram

</br>&emsp;&emsp;[model file] : 이미 생성된 모델 파일명

</br>&emsp;&emsp;[bin model file] : binary로 생성할 모델 파일명

In [ ]:
# 11. 생성된 kenlm 모델 파일 binary  file로 변환

! kenlm/build/bin/build_binary sonagi.arpa sonagi.bin

In [ ]:
# 12. model 이 제대로 생성 되었는지 확인
# ! echo "원하는 문장" | kenlm/build/bin/query [(bin) model file]

! echo "소녀 가 소년 을" | kenlm/build/bin/query sonagi.bin
! echo "소녀 가 소년 을" | kenlm/build/bin/query sonagi.arpa

## Step 4: model을 활용하여 주어진 시퀀스 scoring

### &emsp;>> model.score('여름 이 었 다 . ', bos=True, eos=True)
### &emsp;>> model.score('여름 이 었 ', bos=True, eos=False)

In [ ]:
# 13. 생성한 모델 불러오기

model_file = 'sonagi.bin' 
# model_file = 'sonagi.arpa' # arpa 모델도 사용가능하나, binary 파일이 속도가 좀더 빠름.

model = kenlm.Model(model_file)

In [ ]:
# 14. 시퀀스 scoring 예시

# model.score('주어진 시퀀스', bos=True, eos=True)
# bos : 모델이 scoring하도록 주어진 시퀀스 내에서 sentence가 시작했는가
# eos : 모델이 scoring하도록 주어진 시퀀스 내에서 sentence가 종결되었는가

# 이미 완성한 시퀀스(문장)의 scoring을 위해서는 bos=True eos=True
# bos=True로 하면 모델이 시퀀스의 앞에 <s>를 추가하고,
# eos=True인 경우 모델이 시퀀스의 뒤에 </s>를 추가하여 score를 계산함

#따라서 bos=True, eos=True는 '<s> 원하는 시퀀스 </s>' 의 scoring을 한 것임.


#  "<s> 여름 이 었 다 . </s>" 를 scoring'
eos_score = model.score('여름 이 었 다 .', bos=True, eos=True)
print("<s> 여름 이 었 다 . </s> :", eos_score)

#  "<s> 여름 이 었" 을 scoring
no_eos_score = model.score('여름 이 었', bos=True, eos=False)
print("<s> 여름 이 었 :", no_eos_score)

## Step 5: 주어진 시퀀스에서 다음 형태소 예측

### &emsp;<< Exercise 1>> '소년 은 소녀 를' 다음에 올 가장 자연스러운 형태소를 찾아주세요.

In [ ]:
# 15. vocab list 파일 불러오기

vocab_file = 'sonagi.voc'
vocab_list = []
with open(vocab_file, 'rt', encoding='utf-8') as f:
    for line in f:
        vocab_list.append(line.strip())

In [ ]:
###########################################################################################################################
# 16. << Exercise 1 >> 주어진 시퀀스('소년 은 소녀 를') 다음의 형태소를 예측 : 주어진 시퀀스에 voc list의 형태소 하나를 합쳐서 score를 계산

# 주어진 시퀀스에 preprocessing하며 만든 sonagi.voc 파일의 형태소를 하나씩 붙여서 가장 score가 높은 형태소를 찾음
# voc = ['가', '만', '사업', ...] 이라면,
#'소년 은 소녀 가' 의 점수, '소년 은 소녀 만', '소년 은 소녀 사업 ', ... 등의 score를 계산하고 그 중 가장 높은 score를 지닌 형태소를 찾음
############################################################################################################################

target = "소년 은 소녀 를"
    
best_token = None
best_score = None

for vocab in vocab_list:
    sequence_cand = target + " " + vocab
    score_cand = model.score(sequence_cand, bos=True, eos=False)

    if best_vocab is None:
        best_score = score_cand
        best_vocab = vocab

    if best_score < score_cand:
        best_score = score_cand
        best_vocab = vocab

sequence_cand = target
score_cand = model.score(sequence_cand, bos=True, eos=True)

if best_score < score_cand:
    best_score = score_cand
    best_vocab = '</s>'

print(best_vocab)

## Step 6: 주어진 시퀀스로 시작하는 가장 자연스러운 문장 예측

#### </br>&emsp;<< Exercise 2 >> Step 5와 loop문을 사용하여,
#### </br> &emsp;'소년 은 소녀 를' 으로 시작하는 가장 자연스러운 문장을 찾아주세요.

In [ ]:
###############################################################################################################################
# 17. << Exercise 2 >> 16를 auto-regressive하게 loop 문으로 반복하여 주어진 시퀀스 '소년 은 소녀 를' 로 시작한 문장이 어떻게 끝날지 예측해주세요.

# 1) 주어진 시퀀스에서 문장이 종결될 경우의 score를 계산
# 2) 주어진 시퀀스에서 문장이 종결되지 않을 경우에 그 다음에 올 가장 자연스러운 형태소와 score를 계산
# 3) 만약 1)보다 2)의 score가 더 높다면, 시퀀스에 2)에서 찾아낸 형태소를 추가하고, 다시 1)부터 반복
# 4) 만약 1)의 score가 2)의 score 보다 더 높다면, 그대로 시퀀스를 마무리 함
# find_next_token 함수를 반드시 쓸 필요는 없습니다!
###############################################################################################################################

def find_full_sentence(partial_sentence, model_path, voc_path):

    # load language model with KenLM
    model = kenlm.Model(model_path)
    print("Language Model at " + model_path + " loaded.")

    # load vocabulary (or load word list)
    voc = []
    file_reader = open(voc_path, "r", encoding='utf-8')
    for line in file_reader:
        voc.append(line.strip())
    print("Vocabulary at " + voc_path + " loaded\n")

    # prediction loop
    loop_condition = True
    while loop_condition:
        best_score = None
        best_vocab = None

        for vocab in voc:
            sequence_cand = sequence + " " + vocab
            sequence_score = model.score(sequence_cand, bos=True, eos=False)

            if best_score is None:
                best_score = sequence_score
                best_vocab = vocab

            if best_score < sequence_score:
                best_score = sequence_score
                best_vocab = vocab

        eos_sentence = sequence
        eos_score = model.score(eos_sentence, bos=True, eos=True)
        
        if eos_score < best_score:
            sequence = sequence + " " + best_vocab
      
        else:
            loop_condition = False

    return sequence

In [ ]:
# 17. 16을 실행 

target = input("Sequence: ")
print("Given sequence: " + target)

model_path = 'sonagi.arpa'
voc_path = 'sonagi.voc'
print(find_full_sentence(target, model_path, voc_path))

## Step 7: 다른 말뭉치로 학습한 모델과 비교

#### </br> &emsp;<< Exercise 3 >> Step 1, 2, 3, 4, 6을 참고하여, 
#### </br> &emsp;bucketwheat_flowers.doc를 사용한 모델을 생성하여 
#### </br> &emsp;주어진 시퀀스('소년 은 소녀 를', '개울가 가', '여름 에')로 시작하는 가장 자연스러운 문장들을 찾고, 
#### </br> &emsp;sonagi.doc를 사용한 경우와 비교해주세요.

In [ ]:
####################################################################################################################
# 18. << Exercise 4 >> : bucketwheat_flowers.doc를 사용한 모델을 생성하여 sonagi.doc를 사용한 경우와 주어진 시퀀스에 대한 output을 비교해주세요.
# 주어진 시퀀스: '소년 은 소녀 를', '개울가 가', '여름 에'

# sonata.doc를 data preprocessing하여 sonata.txt, sonata.voc를 생성.
# sonata.txt, sonata.voc를 이용하여 모델을 생성, load하여서
# 같은 문장에 대해서 sonagi corpus를 이용한 경우와 sonata corpus를 이용한 경우에 output이 어떠한지 출력
####################################################################################################################

# Data preprocessing. 형태소 분하여 buckwheat_flowers.txt, buckwheat_flowers.voc 작성
b_lines = []
with open('buckwheat_flowers.doc', 'rt', encoding='utf-8') as b_f:
    for b_line in b_f:
        b_lines.append(b_line)

b_sentences = []
for line in b_lines:
    b_sentences.extend(sent_tokenize(line))

b_morph_sentences = []
for sentence in b_sentences:
    temp = ""
    for morpheme in tokenizer.morphs(sentence):
      temp = temp + " " + morpheme
    b_morph_sentences.append(temp.strip())

with open('buckwheat_flowers.txt', 'wt', encoding='utf-8') as f:
    for line in b_morph_sentences:
        f.write(line.strip())
        f.write('\n')

b_vocab = set()
for sentence in b_morph_sentences:
    temp = sentence.split()
    for t in temp:
        b_vocab.add(t)

with open('buckwheat_flowers.voc', 'wt', encoding='utf-8') as f:
    for line in b_vocab:
        f.write(line.strip())
        f.write('\n')


# n-gram language model을 buckwheat_flowers로 학습

! kenlm/build/bin/lmplz -o 3 <buckwheat_flowers.txt> buckwheat_flowers.arpa
! kenlm/build/bin/build_binary buckwheat_flowers.arpa buckwheat_flowers.bin

In [ ]:
# 19. 18을 실행
target = input("Sequence: ")
print("Given sequence: " + target)

model_path2 = 'buckwheat_flowers.arpa'
voc_path2 = 'buckwheat_flowers.voc'
print(find_full_sentence(target, model_path, voc_path))